# Gradient:
Frechet derivative:
$$\frac{\partial I_0}{\partial m_0} = \lim_{h\rightarrow0} \frac{I(m+h)-I(m)}{h}$$


In [5]:
from firedrake import (
    RectangleMesh,
    FunctionSpace,
    Function,
    SpatialCoordinate,
    conditional,
    File,
)

from firedrake import *
from firedrake_adjoint import *
import spyro
import copy
import numpy as np
from spyro.domains import quadrature
import math
import numpy                  as np
import matplotlib.pyplot      as plot
import matplotlib.ticker      as mticker  
from matplotlib               import cm, ticker
from mpl_toolkits.axes_grid1  import make_axes_locatable

In [6]:
model = {}
model["opts"] = {
    "method": "KMV",  # either CG or KMV
    "quadratrue": "KMV",  # Equi or KMV
    "degree": 2,  # p order
    "dimension": 2,  # dimension
}
model["parallelism"] = {
    "type": "spatial",  # options: automatic (same number of cores for evey processor), custom, off.
    "custom_cores_per_shot": [],  # only if the user wants a different number of cores for every shot.
    "num_cores_per_shot": 1
}
model["mesh"] = {
    "Lz": 1.,  # depth in km - always positive
    "Lx": 1.,  # width in km - always positive
    "Ly": 0.0,  # thickness in km - always positive
    "meshfile": "meshes/square.msh",
    "initmodel": "not_used.hdf5",
    "truemodel": "velocity_models/MarmousiII_w1KM_EXT_GUESS.hdf5",
}
model["BCs"] = {
    "status": False,  # True or false
    "outer_bc": "None",  #  None or non-reflective (outer boundary condition)
    "damping_type": "polynomial",  # polynomial, hyperbolic, shifted_hyperbolic
    "exponent": 2,  # damping layer has a exponent variation
    "cmax": 4.5,  # maximum acoustic wave velocity in PML - km/s
    "R": 1e-6,  # theoretical reflection coefficient
    "lz": 0.0,  # thickness of the PML in the z-direction (km) - always positive
    "lx": 0.0,  # thickness of the PML in the x-direction (km) - always positive
    "ly": 0.0,  # thickness of the PML in the y-direction (km) - always positive
}
model["acquisition"] = {
    "source_type": "Ricker",
    "num_sources": 1,
    "source_pos": spyro.create_transect((-0.1, 0.1), (-0.1, 0.9), 1),
    "frequency": 4.0,
    "delay": 1.0,
    "num_receivers": 100,
    "receiver_locations": spyro.create_transect((-0.10, 0.1), (-0.10, 0.9), 100),
}
model["timeaxis"] = {
    "t0": 0.0,  #  Initial time for event
    "tf": 1.00,  # Final time for event
    "dt": 0.001,
    "amplitude": 1,  # the Ricker has an amplitude of 1.
    "nspool": 100,  # how frequently to output solution to pvds
    "fspool": 99999,  # how frequently to save solution to RAM
}
num_rec = model["acquisition"]["num_receivers"]
δs = np.linspace(0.1, 0.9, num_rec)
X, Y = np.meshgrid(-0.1, δs)
xs = np.vstack((X.flatten(), Y.flatten())).T

In [18]:
# outfile_total_gradient = File(os.getcwd() + "/results/Gradient.pvd")

functional = spyro.utils.compute_functional


def _make_vp_exact(V, mesh):
    """Create a circle with higher velocity in the center"""
    z, x = SpatialCoordinate(mesh)
    vp_exact = Function(V).interpolate(
        4.0
        + 1.0 * tanh(10.0 * (0.5 - sqrt((z - 1.5) ** 2 + (x + 1.5) ** 2)))
        # 5.0 + 0.5 * tanh(10.0 * (0.5 - sqrt((z - 1.5) ** 2 + (x + 1.5) ** 2)))
    )
    File("exact_vel.pvd").write(vp_exact)
    return vp_exact


def _make_vp_exact_pml(V, mesh):
    """Create a half space"""
    z, x = SpatialCoordinate(mesh)
    velocity = conditional(z > -0.5, 1.5, 4.0)
    vp_exact = Function(V, name="vp").interpolate(velocity)
    File("exact_vel.pvd").write(vp_exact)
    return vp_exact


def _make_vp_guess(V, mesh):
    """The guess is a uniform velocity of 4.0 km/s"""
    z, x = SpatialCoordinate(mesh)
    vp_guess = Function(V).interpolate(4.0 + 0.0 * x)
    File("guess_vel.pvd").write(vp_guess)
    return vp_guess


def test_gradient():
    _test_gradient(model)


def test_gradient_pml():
    _test_gradient(model_pml, pml=True)


def _test_gradient(options, pml=False):

    comm = spyro.utils.mpi_init(options)

    mesh, V = spyro.io.read_mesh(options, comm)

    if pml:
        vp_exact = _make_vp_exact_pml(V, mesh)
        z, x = SpatialCoordinate(mesh)
        Lx = model_pml["mesh"]["Lx"]
        Lz = model_pml["mesh"]["Lz"]
        x1 = 0.0
        x2 = Lx
        z1 = 0.0
        z2 = -Lz
        boxx1 = Function(V).interpolate(conditional(x > x1, 1.0, 0.0))
        boxx2 = Function(V).interpolate(conditional(x < Lx, 1.0, 0.0))
        boxz1 = Function(V).interpolate(conditional(z > z2, 1.0, 0.0))
        mask = Function(V).interpolate(boxx1 * boxx2 * boxz1)
        File("mask.pvd").write(mask)
    else:
        vp_exact = _make_vp_exact(V, mesh)

        mask = Function(V).assign(1.0)

    vp_guess = _make_vp_guess(V, mesh)

    sources = spyro.Sources(options, mesh, V, comm)

    wavelet = spyro.full_ricker_wavelet(
        options["timeaxis"]["dt"],
        options["timeaxis"]["tf"],
        options["acquisition"]["frequency"],
    )
    control = Control(vp_exact)
    
    solver  = spyro.solver_AD()
    
    # simulate the guess model
    p_rec_guess = solver.forward_AD(model, mesh, comm,
                               vp_guess, sources, wavelet, xs)
        
    solver.p_true_rec       = p_rec_guess
    solver.Calc_Jfunctional = True
    
    # simulate the exact model
    p_rec_exact  = solver.forward_AD(model, mesh, comm,
                               vp_exact, sources, wavelet, xs)

    J = solver.obj_func
   
    dJ = compute_gradient(J, control)
    Jm = copy.deepcopy(J)

    qr_x, _, _ = quadrature.quadrature_rules(V)

    print("\n Cost functional at fixed point : " + str(Jm) + " \n ")

      
    dJ *= mask
    File("gradient.pvd").write(dJ)

    steps = [1e-3, 1e-4, 1e-5]   # , 1e-6]  # step length

    delta_m = Function(V)  # model direction (random)
    delta_m.assign(dJ)

    # this deepcopy is important otherwise pertubations accumulate
    vp_original = vp_guess.copy(deepcopy=True)

    errors = []
    for step in steps:  # range(3):
        # steps.append(step)
        # perturb the model and calculate the functional (again)
        # J(m + delta_m*h)
        solver.obj_func   = 0.
        solver.p_true_rec = p_rec_exact
        vp_guess = vp_original +  step* delta_m 
        p_rec_guess = solver.forward_AD(model, mesh, comm,
                               vp_guess, sources, wavelet, xs)

        Jp = solver.obj_func
        projnorm = assemble(mask * dJ * dx(rule=qr_x))

        fd_grad = (Jp - Jm) / step
        print(
            "\n Cost functional for step "
            + str(step)
            + " : "
            + str(Jp)
            + ", fd approx.: "
            + str(fd_grad)
            + ", grad'*dir : "
            + str(projnorm)
            + " \n ",
        )

        errors.append(100 * ((fd_grad - projnorm) / projnorm))
        # step /= 2
        

    # all errors less than 1 %
    errors = np.array(errors)
    print(errors)
    assert (np.abs(errors) < 5.0).all()


test_gradient()

INFO: Distributing 1 shot(s) across 1 core(s). Each shot is using 1 cores
  rank 0 on ensemble 0 owns 780 elements and can access 431 vertices
Simulation time is:        0.0 seconds
Simulation time is:      0.099 seconds
Simulation time is:      0.199 seconds
Simulation time is:      0.299 seconds
Simulation time is:      0.399 seconds
Simulation time is:      0.499 seconds
Simulation time is:      0.599 seconds
Simulation time is:      0.699 seconds
Simulation time is:      0.799 seconds
Simulation time is:      0.899 seconds
---------------------------------------------------------------
Simulation time is:        0.0 seconds
Simulation time is:      0.099 seconds
Simulation time is:      0.199 seconds
Simulation time is:      0.299 seconds
Simulation time is:      0.399 seconds
Simulation time is:      0.499 seconds
Simulation time is:      0.599 seconds
Simulation time is:      0.699 seconds
Simulation time is:      0.799 seconds
Simulation time is:      0.899 seconds
-------------

AssertionError: 